# QAtools Notebook
[![DOI:<10.26023>](http://img.shields.io/badge/DOI-10.26023-green.svg)](https://doi.org/10.26023/a0e3-4n78)

Quick-look statistics and plots from field-phase NCAR/RAF flight netCDF datasets.

## Table of Contents
* [References](#references)
* [Project and Flight Data information](#project_and_flight_data_information)
* [Flight Track](#flight_track)
* [Altitude, Heading, and GPS Quality](#alt_heading_gps)
* [Temperature](#temperature)
    * [Air Temperature](#air_temperature)
    * [Dew Point Temperature](#dew_point_temperature)
* [Pressure](#pressure)
    * [Cavity Pressure](#cavity_pressure)
    * [Static Pressure](#static_pressure)
    * [Dynamic Pressure](#dynamic_pressure)
* [Airspeed](#airspeed)
* [Wind](#wind)
    * [Wind Direction](#wind_direction)
    * [Wind Speed](#wind_speed)
* [Schuler Oscillation](#schuler_oscillation)
* [Radome Pressures and Angles](#radome)
* [Pitch Roll Heading](#pitch_roll_heading)
    * [Pitch](#pitch)
    * [Roll](#roll)
    * [Heading](#heading)
* [ACINS VSPD Altitude](#acins_vspd_altitude)
    * [ACINS](#acins)
    * [VSPD](#vspd)
    * [Altitude](#altitude)
* [Concentrations](#concentrations)
* [Liquid Water](#liquid_water)
* [Skew-T Plot](#skewt)
* [Potential Temperature](#potential_temperature)
* [Chemistry](#chemistry)
* [Aerosol and Cloud Probe Histogram Data](#histogram) 
   
    

## Change Log

Version 0.1 7/18/2023:
Initial Release 
GitHub Repo: https://github.com/NCAR/aircraft_QAtools_notebook

## References <a class="anchor" id="references"></a>

README: https://github.com/NCAR/aircraft_QAtools_notebook/blob/main/README.md

NCAR/RAF netCDF Format: https://archive.eol.ucar.edu/raf/software/netCDF.html

NCAR/RAF Processing Algorithms: https://ncar.github.io/aircraft_ProcessingAlgorithms/

RSIG Project QAQC Google Sheet: https://drive.google.com/drive/folders/1HB5dQ867T62jWOrqCDD3BV1B6Xiu2w7Q  

## Project and Flight Data Information <a class="anchor" id="project_and_flight_data_information"></a>

In [ ]:
import os
try:
    os.system('source ~/.bashrc')
    CL = os.environ.get("QA_CL")
except:
    pass

In [ ]:
if CL == 'command_line_mode':
    try:
        project = os.environ.get('QA_PROJ')
        flight = os.environ.get('QA_FLIGHT')
    except:
        pass
else:
    try:
        #######################################################################
        project = 'ti3ger'
        flight = 'rf05'
        #######################################################################
    except:
        pass

In [ ]:
print('Project: ' + project)
print('Flight: ' + flight)
interactive_histogram = False

In [ ]:
import netCDF4
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize, ListedColormap
from metpy.plots import SkewT
from metpy.units import pandas_dataframe_to_unit_arrays, units
from datetime import datetime, timedelta
from IPython.display import display
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import Title, CustomJS, Select, TextInput, Button, LinearAxis, Range1d, FuncTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.palettes import Category10
import warnings
import itertools 
import holoviews as hv 
from holoviews import dim, opts
import hvplot.pandas
hv.extension('bokeh', 'matplotlib')
warnings.filterwarnings('ignore')
output_notebook()
%matplotlib inline

In [ ]:
# function definition for creating generic timeseries plot
def timeseries_plot(title, y_label, dataframe):
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title=title) 
    plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
    plot.add_layout(Title(text=y_label, align="center"), "left")
    
    # loop over variables in the pandas dataframe to add to plot
    for i in asc[dataframe].columns:
        try:
            plot.line(dtime, asc[dataframe][i], color=next(colors), legend_label=i)
            plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
        except Exception as e:
            print(e)
        try:
            plot.legend.location = "top_left"
        except Exception as e:
            print(e)
    try:
        show(plot)
    except Exception as e:
        print(e)

In [ ]:
# use the user provided project and flight information to build the path and store the data file as an object
filepath = '/Users/taylort/aircraft_QAtools_notebook/'
input_file = project + flight + '.nc'
nc = netCDF4.Dataset(filepath + '/' + project + '/' + input_file, mode='r')

# try to get global attributes from the netcdf file if they are present
# determine preliminary or final status
try:
    proc_status = nc.getncattr('WARNING')
    print(proc_status)
except:
    proc_status = 'final'

# determine the NIDAS version
try:
    nidas = nc.getncattr('NIDASrevision')
    print('NIDAS version: ' + nidas)
except Exception as e:
    print(e)

# determine the NIMBUS version
try:
    nimbus = nc.getncattr('RepositoryRevision')
    print('NIMBUS version: ' + nimbus)
except Exception as e:
    print(e)

# determine the processing date and time
try:
    proc_date = nc.getncattr('date_created')
    print('Processing Date & Time: ' + proc_date)
except Exception as e:
    print(e)

In [ ]:
# sometimes the netcdf4 api produces an issue with big-endian buffer on little-endian compiler
byte_swap = False

# create empty placeholders for asc, histo_asc and units
asc = {}
histo_asc = {}
units = {}
cellsize = {}

# use the netcdf4 api to get the netcdf data into a dataframe
try:
    
    # loop over keys in netCDF file and organize
    for i in nc.variables.keys():
        dims = str(nc.variables[i].dimensions)
        
        # this if section retrieves data that has time dimension only
        if dims == "('Time',)":
            output = nc[i][:]
            # sometimes the netcdf4 api produces an issue with big-endian buffer on little-endian compiler
            if byte_swap == True:
                output = output.byteswap().newbyteorder()
            else:
                pass
            asc[i] = pd.DataFrame(output)
            units_var = nc.variables[i].getncattr('units')
            units[i] = pd.Series(units_var)
            asc[i].columns = pd.MultiIndex.from_tuples(zip(asc[i].columns, units[i]))
        
        # this section retrieves data that has a size distribution dimension in addition to time
        elif "sps1" in dims:
            histo_output = nc.variables[i][:, 0, :]
            # sometimes the netcdf4 api produces an issue with big-endian buffer on little-endian compiler
            if byte_swap == True:
                histo_output = histo_output.byteswap().newbyteorder()
            else:
                pass
            histo_asc[i] = pd.DataFrame(histo_output)
            
            # this try / except block accommodates size distribution data that has an attribute CellSizes
            try:
                cellsize = nc.variables[i].getncattr('CellSizes')
                histo_asc[i].columns = pd.MultiIndex.from_tuples(zip(histo_asc[i].columns, cellsize))
            except Exception as e:
                histo_asc[i].columns = pd.MultiIndex.from_tuples(zip(histo_asc[i].columns, histo_asc[i].columns))
        else:
            pass    

    # concatenate the dataframe
    asc = pd.concat(asc, axis=1, ignore_index=False)
    
    # extract the Time variable and organize as a pandas series
    dtime = nc.variables['Time']
    dtime = pd.DataFrame(dtime[:])
    new_dtime = []
    for index in range(len(dtime)):
        new_dtime.append(timedelta(seconds=int(dtime.loc[index])))
    dtime = pd.Series(new_dtime)
    
    # clean up the dataframe by dropping some of the multi-index rows
    asc.columns = asc.columns.droplevel(1)
    asc.columns = asc.columns.droplevel(1)
    
    # concatenate the histogram data
    histo_asc = pd.concat(histo_asc, axis=1, ignore_index=False)
    histo_asc.columns = histo_asc.columns.droplevel(1)
    colors = itertools.cycle(Category10[6])
    
    # scrub data based on ground speed change value from 60 to something else if you want more or less data
    asc.drop(asc[asc['GGSPD'] < 60].index, inplace = True)
except Exception as e:
    print(e)

## Flight Track <a class="anchor" id="flight_track"></a>

In [ ]:
# get latitude and longitude from asc dataframe
latitude = asc["GGLAT"]
longitude = asc["GGLON"]

# update to mercator projection
k = 6378137
longitude = longitude * (k * np.pi/180.0)
latitude = np.log(np.tan((90 + latitude) * np.pi/360.0)) * k

# create the plot layout and add axis labels
try:
    plot = figure(width=1000, height=1000, title=project + ' ' + flight + " Flight Track", x_axis_type="mercator", y_axis_type="mercator") 
    plot.add_layout(Title(text="Longitude [Degrees]", align="center"), "below")
    plot.add_layout(Title(text="Latitude [Degrees]", align="center"), "left")
except Exception as e:
    print(e)

# add the flight track in yellow and add Esri World Imagery as the basemap
try:
    plot.line(longitude,latitude, color="yellow")
    plot.add_tile("Esri World Imagery", retina=True)
    show(plot)
except Exception as e:
    print(e)

## Altitude, Heading, and GPS Quality <a class="anchor" id="alt_heading_gps"></a>

In [ ]:
# create a subset for altitude in order to generate stats
alt_heading = []
for i in asc.columns:
    if i.startswith('GGALTF') | i.startswith('PALTF') | i.startswith('PA2'):
        alt_heading.append(i)

# generate the stats table
asc[alt_heading].describe()

In [ ]:
# generate the altitude, heading and gps quality plots
# altitude plot
timeseries_plot("Altitude", "Altitude [Feet]", alt_heading)
colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="Heading") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="True Heading [Degrees]", align="center"), "left")
try:
    plot.line(dtime, asc['THDG'], color=next(colors))
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    show(plot)
except:
    pass

colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="GPS Quality: 5 = TerraStar corrected, 2 = TerraStar converging, 9 = WAAS, 1 = Standard GPS, 0 = No Fix") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="GPS Quality []", align="center"), "left")
try:
    plot.line(dtime, asc['GGQUAL'], color=next(colors), legend_label="GGQUAL")
except:
    pass
try:
    plot.line(dtime, asc['GGNSAT'], color=next(colors), legend_label="GGNSAT")
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
except:
    pass
try:
    show(plot)
except:
    pass

## Temperature <a class="anchor" id="temperature"></a>

### Air Temperature <a class="anchor" id="air_temperature"></a>

In [ ]:
# create a subset for air temperature in order to generate stats
temp = []
for i in asc.columns:
    if i.startswith('ATX') | i.startswith('ATH') | i.startswith('ATF') | i.startswith('AT_'):
        temp.append(i)
# remove data that is not useful
try:
    temp.remove('AT_VXL')
except:
    pass
    
# generate the stats table
try:
    asc[temp].describe()
except:
    pass

In [ ]:
timeseries_plot("Temperature", "Temperature [\N{DEGREE CELSIUS}]", temp)
    
colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="ATX - Temperature") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Temperature Difference [\N{DEGREE CELSIUS}]", align="center"), "left")
for i in asc[temp].columns:
    try:
        plot.line(dtime, asc[temp][i] -  asc['ATX'], color=next(colors), legend_label=i)
        plot.line(dtime, 0.25, color = 'black', line_dash="4 4", line_width=2)
        plot.line(dtime, -0.25, color = 'black', line_dash="4 4", line_width=2)

        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass
    
# loop over temperatures
for i in asc[temp].columns:    
    try:
        # create plot labels
        plot = figure(width=700, height=500, title=asc[temp][i].name) 
        plot.add_layout(Title(text="ATH1 [\N{DEGREE CELSIUS}]", align="center"), "below")
        plot.add_layout(Title(text=asc[temp][i].name + " [\N{DEGREE CELSIUS}]", align="center"), "left")
    except:
        pass
    try:
        # create scatter plots
        plot.scatter(asc[temp]['ATX'], asc[temp]['ATX'], color = 'black', line_dash = 'dotted', legend_label='1:1')
        plot.scatter(asc[temp]['ATH1'], asc[temp][i], color='orange', legend_label=i)
    except:
        pass
    try:
        # create additional y axis range
        plot.extra_y_ranges['delta'] = Range1d(-4, 4)
        ax2 = LinearAxis(y_range_name="delta", axis_label="Delta Temperature [C]")
        ax2.axis_label_text_color ="red"
        plot.add_layout(ax2, 'right')
    except:
        pass
    try:
        # create line plot of temperature differences using the additional y axis range
        plot.line(asc[temp]['ATH1'], asc[temp][i]-asc[temp]['ATH1'], color='red', legend_label=i, y_range_name="delta")
        plot.line(asc[temp]['ATH1'], 1, color = 'gray', y_range_name="delta")
        plot.line(asc[temp]['ATH1'], - 1, color = 'gray', y_range_name="delta")
    except:
        pass
    try:
        plot.legend.location = "top_left"
    except:
        pass
    show(plot)

### Dew Point Temperature <a class="anchor" id="dew_point_temperature"></a>

In [ ]:
# create a subset for dew point temperature in order to generate stats
dpt = []
for i in asc.columns:
    if i.startswith('DP_') | i.startswith('ATX'):
        dpt.append(i)
asc[dpt].describe()


In [ ]:
timeseries_plot("Dew Point Temperatures", "Dew Point [C]", dpt)

colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="Diff Dew Point Temperatures") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Diff DPL - DP Dew Point [C]", align="center"), "left")
for i in asc[dpt].columns:
    try:
        plot.line(dtime, asc['DP_DPL'] - asc[dpt][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass

try:
    plot.line(dtime, 1, color = 'black', line_dash="4 4", line_width=2)
    plot.line(dtime, -1, color = 'black', line_dash="4 4", line_width=2)
    plot.legend.location = "top_left"
    plot.y_range=Range1d(-5, 5)
    show(plot)
except:
    pass 

In [ ]:
# create plot labels
plot = figure(width=1200, height=800) 
plot.add_layout(Title(text="DP_DPR [C]", align="center"), "below")
plot.add_layout(Title(text="DP_DPL [C]", align="center"), "left")
try:
    try:
        # create scatter plots
        plot.scatter(asc['DP_DPR'], asc['DP_DPL'], color = 'blue', legend_label='DP_DPL')
    except:
        pass    
    try:
        # DP_DPR used to create a 1:1 line
        plot.line(asc['DP_DPR'], asc['DP_DPR'], color = 'black', legend_label='1:1')
        # +- 5C lines for quick visualization on bounds
        plot.line(asc['DP_DPR'], asc['DP_DPR']+5, color = 'orange', legend_label='+-5C')
        plot.line(asc['DP_DPR'], asc['DP_DPR']-5, color = 'orange')
    except:
        pass
    try:
        plot.scatter(asc['DP_DPR'], asc['DP_VXL'], color = 'green', legend_label='DP_VXL')
    except:
        pass
except:
    pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass


## Pressure <a class="anchor" id="pressure"></a>

### Cavity Pressure <a class="anchor" id="cavity_pressure"></a>

In [ ]:
# create a subset for cavity pressure in order to generate stats
cav_pressure = []
for i in asc.columns:
    if i.startswith('CAVP_') | i.startswith('CAVPF_') | i.startswith('PSXC'):
        cav_pressure.append(i)
# remove some vars that don't provide value
try:
    cav_pressure.remove('CAVP_WVISO2346')
except:
    pass
try:
    cav_pressure.remove('CAVP_GNI')
except:
    pass
# show the stats table
try:
    asc[cav_pressure].describe()
except:
    pass

In [ ]:
timeseries_plot("Cavity Pressure", "Pressure [hPa]", cav_pressure)

### Vapor Pressure <a class="anchor" id="vapor_pressure"></a>

In [ ]:
vapor_pressure = []
for i in asc.columns:
    if i.startswith('EW_'):
        vapor_pressure.append(i)
try:
    asc[vapor_pressure].describe()
except:
    pass

#mixing_ratio = []
#for i in asc.columns:
#    if i.startswith('MR_'):
#        mixing_ratio.append(i)
#try:
#   asc[mixing_ratio].describe()
#except:
#    pass
    
rh = []
for i in asc.columns:
    if i.startswith('RH_'):
        rh.append(i)
asc[rh].describe()

In [ ]:
timeseries_plot("Vapor Pressure", "EWy [hPa]", vapor_pressure)
timeseries_plot("Relative Humidity", "RH [%]", rh)

### Static Pressure <a class="anchor" id="static_pressure"></a>

In [ ]:
static_pressure = []
for i in asc.columns:
    if i.startswith('PS_') | i.startswith('PSF') | i.startswith('PST'):
        static_pressure.append(i)
    
asc[static_pressure].describe()


In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Static Pressure") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Pressure [hPa]", align="center"), "left")
for i in asc[static_pressure].columns:
    try:
        plot.line(dtime, asc[static_pressure][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus PSFC") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Pressure - PSFC [C]", align="center"), "left")
for i in asc[static_pressure].columns:
    try:
        plot.line(dtime, asc[static_pressure][i] - asc['PSFC'], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
plot.line(dtime, 1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
plot.line(dtime, -1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
try:
    plot.legend.location = "top_left"
    plot.y_range=Range1d(-5, 5)
    show(plot)
except:
    pass
    
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus PSXC") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Pressure - PSXC [C]", align="center"), "left")
for i in asc[static_pressure].columns:
    try:
        plot.line(dtime, asc['PSXC'] - asc[static_pressure][i], color=next(colors), legend_label=i)
    except:
        pass

plot.line(dtime, 1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
plot.line(dtime, -1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
try:
    plot.legend.location = "top_left"
    plot.y_range=Range1d(-10, 10)
    show(plot)
except:
    pass

### Dynamic Pressure <a class="anchor" id="dynamic_pressure"></a>

In [ ]:
dynamic_pressure_corr = []
for i in asc.columns:
    if i.startswith('QC_') | i.startswith('QCTFC') | i.startswith('QCRC') | i.startswith('QCFC'):
        dynamic_pressure_corr.append(i)
asc[dynamic_pressure_corr].describe()

In [ ]:
dynamic_pressure_uncorr = []
for i in asc.columns:
    if i.startswith('QC_') | i.startswith('QCTF') | i.startswith('QCR') | i.startswith('QCF'):
        dynamic_pressure_uncorr.append(i)
try:
    dynamic_pressure_uncorr.remove('QCRTEMP')
except:
    pass
try:
    dynamic_pressure_uncorr.remove('QCFTEMP')
except:
    pass
try:
    dynamic_pressure_uncorr.remove('QCTFTEMP')
except:
    pass
try:
    dynamic_pressure_uncorr.remove('QCTF')
except:
    pass
try:
    asc[dynamic_pressure_uncorr].describe()
except:
    pass

In [ ]:
# plot dynamic pressure timeseries
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Dynamic Pressure") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Corrected Pressure QCyC [hPa]", align="center"), "left")
for i in asc[dynamic_pressure_corr].columns:
    try:
        plot.line(dtime, asc[dynamic_pressure_corr][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

# plot delta pressure QCFC
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus QCFC") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Pressure - QCFC [C]", align="center"), "left")
for i in asc[dynamic_pressure_corr].columns:
    try:
        plot.line(dtime, asc[dynamic_pressure_corr]['QCFC'] - asc[dynamic_pressure_corr][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
plot.line(dtime, 1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
plot.line(dtime, -1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
try:
    plot.legend.location = "top_left"
    plot.y_range=Range1d(-20, 20)
    show(plot)
except:
    pass
# plot delta pressure QCXC
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus QCXC") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Pressure - QCXC [C]", align="center"), "left")
for i in asc[dynamic_pressure_uncorr].columns:
    try:
        plot.line(dtime, asc['QCXC'] - asc[dynamic_pressure_uncorr][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
plot.line(dtime, 1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
plot.line(dtime, - 1, color = 'black', line_dash="4 4", line_width=2)
plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
try:
    plot.legend.location = "top_left"
    plot.y_range=Range1d(-10, 10)
    show(plot)
except:
    pass

## Airspeed <a class="anchor" id="airspeed"></a>

In [ ]:
airspeed = []
for i in asc.columns:
    if i.startswith('TAS'):
        airspeed.append(i)
try:
    airspeed.remove('TASFLG')
except:
    pass
try:
    airspeed.remove('TAS_RWOO')
except:
    pass
asc[airspeed].describe()

In [ ]:
# plot airspeed
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Airspeed") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="TASy [m/s]", align="center"), "left")
for i in asc[airspeed].columns:
    try:
        plot.line(dtime, asc[airspeed][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass

try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass
# plot delta airspeed
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Delta Airspeed") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Difference v TASF [m/s]", align="center"), "left")
for i in asc[airspeed].columns:
    try:
        plot.line(dtime, asc[airspeed][i]-asc['TASF'], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    plot.line(dtime, 1, color = 'black', line_dash="4 4", line_width=2)
    plot.line(dtime, -1, color = 'black', line_dash="4 4", line_width=2)
    plot.y_range=Range1d(-3, 3)
    show(plot)
except:
    pass

In [ ]:
mach = []
for i in asc.columns:
    if i.startswith('MACH'):
        mach.append(i)
asc[mach].describe()

In [ ]:
# plot mach
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Mach") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="MACHy", align="center"), "left")
for i in asc[mach].columns:
    try:
        plot.line(dtime, asc[mach][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

In [ ]:
pst = []
for i in asc.columns:
    if i.startswith('PST'):
        pst.append(i)
asc[pst].describe()

In [ ]:
timeseries_plot("Corrected Static Pressure Top of Fuselage", "PSTF(C) [hPa]", pst)

## Wind <a class="anchor" id="wind"></a>

### Wind Direction <a class="anchor" id="wind_direction"></a>

In [ ]:
wd_winds = []
for i in asc.columns:
    if i.startswith('WD') | i.startswith('IWD'):
        wd_winds.append(i)
asc[wd_winds].describe()

In [ ]:
timeseries_plot("Wind Direction", "Wind Speed [m/s]", wd_winds)

### Wind Speed <a class="anchor" id="wind_speed"></a>

In [ ]:
ws_winds = []
for i in asc.columns:
    if i.startswith('WS') | i.startswith('IWS'):
        ws_winds.append(i)
asc[ws_winds].describe()

In [ ]:
timeseries_plot("Wind Speed", "Wind Speed [m/s]", ws_winds)

In [ ]:
wi_winds = []
for i in asc.columns:
    if i.startswith('WIX') | i.startswith('WIC') | i.startswith('WIY'):
        wi_winds.append(i)
asc[wi_winds].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Vertical Wind Speed") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Vertical Wind [m/s]", align="center"), "left")
for i in asc[wi_winds].columns:
    try:
        plot.line(dtime, asc[wi_winds][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
        plot.line(dtime, 2.5, color = 'black', line_dash="4 4", line_width=2)
        plot.line(dtime, -2.5, color = 'black', line_dash="4 4", line_width=2)
        plot.line(dtime, 0, color = 'red', line_dash="4 4", line_width=2)
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

In [ ]:
east_wind = []
for i in asc.columns:
    if i.startswith('UIC') | i.startswith('IUX'):
        east_wind.append(i)
try:
    asc[east_wind].describe()
except:
    pass

In [ ]:
north_wind = []
for i in asc.columns:
    if i.startswith('VIC') | i.startswith('IVY'):
        north_wind.append(i)
asc[north_wind].describe()

In [ ]:
timeseries_plot("East Wind Speed", "East Wind Speed [m/s]", east_wind)
timeseries_plot("North Wind Speed", "North Wind Speed [m/s]", north_wind)

## Schuler Oscillation <a class="anchor" id="schuler_oscillation"></a>

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="IRU Schuler oscillation, east west component ") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="GGVEW [m/s]", align="center"), "left")
try:
    plot.line(dtime, asc["GGVEW"], color=next(colors), legend_label="GGVEW")
except:
    pass
try:
    plot.line(dtime, asc["VEW"], color=next(colors), legend_label="VEW")
except:
    pass  

try:
    plot.line(dtime, (asc["GGVEW"] - asc["VEW"]) *50, color=next(colors), legend_label="Difference * 50")
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
except:
    pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="IRU Schuler oscillation, north south component ") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="GGVNS [m/s]", align="center"), "left")
try:
    plot.line(dtime, asc["GGVNS"], color=next(colors), legend_label="GGVEW")
except:
    pass
try:
    plot.line(dtime, asc["VNS"], color=next(colors), legend_label="VEW")
except:
    pass 
try:
    plot.line(dtime, (asc["GGVNS"] - asc["VNS"]) *50, color=next(colors), legend_label="Difference * 50")
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
except:
    pass 
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

## Radome Pressures and Angles <a class="anchor" id="radome"></a>

In [ ]:
radome_angles = []
for i in asc.columns:
    if i.startswith('AKRD') | i.startswith('PITCH') | i.startswith('AOAREF'):
        radome_angles.append(i)
asc[radome_angles].describe()

In [ ]:
timeseries_plot("Radome Angles", "Angle [Degrees]", radome_angles)

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="SSRD [Degrees]", align="center"), "left")
try:
    plot.line(dtime, asc["SSRD"], color=next(colors), legend_label="SSRD")
except:
    pass
try:
    plot.line(dtime, asc["SSREF"], color=next(colors), legend_label="SSREF")
except:
    pass
try:
    plot.legend.location = "top_left"
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    plot.line(dtime, 2.0, color = 'black', line_dash="4 4", line_width=2)
    plot.line(dtime, -2.0, color = 'black', line_dash="4 4", line_width=2)
    show(plot)
except:
    pass
    
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="ADIFR [hPa]", align="center"), "left")
try:
    plot.line(dtime, asc["ADIFR"], color=next(colors), legend_label="ADIFR")
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
except:
    pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass
    
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="BDIFR [hPa]", align="center"), "left")
try:
    plot.line(dtime, asc["BDIFR"], color=next(colors), legend_label="BDIFR")
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
except:
    pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

## Pitch Roll Heading <a class="anchor" id="pitch_roll_heading"></a>

### Pitch <a class="anchor" id="pitch"></a>

In [ ]:
pitch = []
for i in asc.columns:
    if i.startswith('PITCH'):
        pitch.append(i)
asc[pitch].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Pitch") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="PITCH [Degrees]", align="center"), "left")
for i in asc[pitch].columns:
    try:
        plot.line(dtime, asc[pitch][i], color=next(colors), legend_label=i)
    except:
        pass
    try:
        plot.line(dtime, (asc[pitch]['PITCH'] - asc[pitch]['PITCH_IRS2']) * 50, color=next(colors), legend_label="PITCH - PITCH_IRS2 Difference * 50")
    except Exception as e:
        pass
    try:
        plot.line(dtime, (asc[pitch]['PITCH'] - asc[pitch]['PITCH_IRS3']) * 50, color=next(colors), legend_label="PITCH - PITCH_IRS3 Difference * 50")
    except Exception as e:
        pass
    try:
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass

try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

### Roll <a class="anchor" id="roll"></a>

In [ ]:
roll = []
for i in asc.columns:
    if i.startswith('ROLL'):
        roll.append(i)
try:
    asc[roll].describe()
except:
    pass

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Roll") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="PITCH [Degrees]", align="center"), "left")
for i in asc[roll].columns:
    try:
        plot.line(dtime, asc[roll][i], color=next(colors), legend_label=i)
    except:
        pass
    try:
        plot.line(dtime, (asc[roll]['ROLL'] - asc[roll]['ROLL_IRS2']) * 500, color=next(colors), legend_label="ROLL - ROLL_IRS2 Difference * 500")
    except:
        pass
    try:
        plot.line(dtime, (asc[roll]['ROLL'] - asc[roll]['ROLL_IRS3']) * 500, color=next(colors), legend_label="ROLL - ROLL_IRS3 Difference * 500")
    except:
        pass
    try:
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

### Heading <a class="anchor" id="heading"></a>

In [ ]:
thdg = []
for i in asc.columns:
    if i.startswith('THDG'):
        thdg.append(i)
asc[thdg].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="True Heading") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="THDG [degrees]", align="center"), "left")

for i in asc[thdg].columns:
    try:
        plot.line(dtime, asc[thdg][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass  

## ACINS VSPD Altitude <a class="anchor" id="acins_vspd_altitude"></a>

### ACINS <a class="anchor" id="acins"></a>

In [ ]:
acins = []
for i in asc.columns:
    if i.startswith('ACINS'):
        acins.append(i)

asc[acins].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="ACINS") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="ACINS [m/s^2]", align="center"), "left")

for i in asc[acins].columns:
    try:
        plot.line(dtime, asc[acins][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass  

### VSPD <a class="anchor" id="vspd"></a>

In [ ]:
vspd = []
for i in asc.columns:
    if i.startswith('VSPD'):
        vspd.append(i)
try:
    asc[vspd].describe()
except:
    pass

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="VSPD") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="VSPD", align="center"), "left")

for i in asc[vspd].columns:
    try:
        plot.line(dtime, asc[vspd][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

### Altitude <a class="anchor" id="altitude"></a>

In [ ]:
alt = []
for i in asc.columns:
    if i.startswith('GGALT') | i.startswith('ALT_'):
        alt.append(i)
try:
    alt.remove('GGALTSD_232')
except:
    pass
try:
    alt.remove('GGALTSD')
except:
    pass
try:
    alt.remove('GGALTSD_VTB')
except:
    pass
try:
    alt.remove('GGALTSD_VTB2')
except:
    pass
asc[alt].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Altitude") 
plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
plot.add_layout(Title(text="Altitude (ALT) [m]", align="center"), "left")

for i in asc[alt].columns:
    try:
        plot.line(dtime, asc[alt][i], color=next(colors), legend_label=i)
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except:
        pass
try:
    plot.legend.location = "top_left"
    show(plot)
except:
    pass

## Concentrations <a class="anchor" id="concentrations"></a>

In [ ]:
conc = []
for i in asc.columns:
    if i.startswith('CONCN') | i.startswith('CONCP') | i.startswith('CONCD') | i.startswith('CONC1DC'):
        conc.append(i)
try:
    conc.remove('CONCN')
except:
    pass
asc[conc].describe()

In [ ]:
try:
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title="Concentrations") 
    plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
    plot.add_layout(Title(text="CONCy [cm-3]", align="center"), "left")

    for i in asc[conc].columns:
        try:
            plot.line(dtime, asc[conc][i], color=next(colors), legend_label=i)
            plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
        except:
            pass
    try:
        plot.legend.location = "top_left"
        show(plot)
    except:
        pass  

except:
    pass

try:
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title="CONCN") 
    plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
    plot.add_layout(Title(text="CONCN [cm-3]", align="center"), "left")

    try:
        plot.line(dtime, asc['CONCN'], color=next(colors), legend_label='CONCN')
    except:
        pass
    try:
        plot.line(dtime, asc['CONCU'], color=next(colors), legend_label='CONCU')
    except:
        pass
    try:
        plot.legend.location = "top_left"
    except:
        pass  
    plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    show(plot)
except:
    pass

## Liquid Water <a class="anchor" id="liquid_water"></a>

In [ ]:
liquid_water = []
for i in asc.columns:
    if i.startswith('PLWC') | i.startswith('RICE'):
        liquid_water.append(i)
asc[liquid_water].describe()


In [ ]:
timeseries_plot("Liquid Water", "PLWCy []", liquid_water)

## Skew-T Plot <a class="anchor" id="skewt"></a>

In [ ]:
fig = plt.figure(figsize=(11, 13))
p = asc['PSXC']
T = asc['ATX']
try:
    Td = asc['DP_DPL']
except:
    Td = asc['DP_DPT']
skew = SkewT(fig, rotation=45)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
try:
    skew.plot(p, T, 'r', label='T')
    skew.plot(p, Td, 'g', label='Td')
    # Set some better labels than the default
except:
    pass

try:
    skew.ax.set_xlabel('Temperature or Dew Point [\N{DEGREE CELSIUS}]')
    skew.ax.set_ylabel('Pressure [hPa]')
except:
    pass
# Add the relevant special lines
try:
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-60, 30)
    skew.ax.legend()
    plt.show()
except:
    pass

## Potential Temperature <a class="anchor" id="potential_temperature"></a>

In [ ]:
theta = []
for i in asc.columns:
    if i.startswith('THETA'):
        theta.append(i)

asc[theta].describe()

In [ ]:
timeseries_plot("Potential Temperature", "THETA [K]", theta)

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=800, title="") 
plot.add_layout(Title(text="Potential Temperature [K]", align="center"), "below")
plot.add_layout(Title(text="Pressure [hPa]", align="center"), "left")

for i in asc[theta].columns:
    try:
        plot.line(asc[theta][i], asc['PSXC'], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
plot.y_range.flipped = True
show(plot)

## Chemistry <a class="anchor" id="chemistry"></a>

In [ ]:
try:
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title="Carbon Dioxide") 
    plot.add_layout(Title(text="ppmv", align="center"), "left")

    try:
        plot.line(dtime, asc['CO2C_PIC2401'], color=next(colors), legend_label='CO2C_PIC2401')
    except Exception as e:
        pass
    try:
        plot.line(dtime, asc['CO2_PIC2401'], color=next(colors), legend_label='CO2_PIC2401')
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except Exception as e:
        pass
    try:
        plot.legend.location = "top_left"
    except Exception as e:
        pass  
    show(plot)
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title="Methane") 
    plot.add_layout(Title(text="ppmv", align="center"), "left")

    try:
        plot.line(dtime, asc['CH4C_PIC2401'], color=next(colors), legend_label='CH4C_PIC2401')
    except Exception as e:
        pass
    try:
        plot.line(dtime, asc['CH4_PIC2401'], color=next(colors), legend_label='CH4_PIC2401')
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except Exception as e:
        pass
    try:
        plot.legend.location = "top_left"
    except Exception as e:
        pass  
    show(plot)
    
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title="Carbon Monixide and Fast Ozone (if available, scaled / 1000)") 
    plot.add_layout(Title(text="Time [UTC]", align="center"), "below")
    plot.add_layout(Title(text="ppb", align="center"), "left")
    try:
        plot.line(dtime, asc['CO_PIC2401'], color=next(colors), legend_label='CO_PIC2401')
    except Exception as e:
        pass
    try:
        plot.line(dtime, asc['CO_ARI']/1000, color=next(colors), legend_label='CO_ARI')
        plot.xaxis.formatter=DatetimeTickFormatter(hourmin = ['%H:%M'])
    except Exception as e:
        pass
    try:
        plot.legend.location = "top_left"
    except:
        pass  
    show(plot)
except:
    pass

## Aerosol and Cloud Probe Histogram Data <a class="anchor" id="histogram"></a>

In [ ]:
# create lists of the histogram variables
try:
    histovar_list = list(histo_asc.columns.levels[0])
    histovar_list = [var for var in histovar_list if not var.endswith('VXL')]
    histobin_list = []
except:
    pass

# loop over the length of the histogram variable list
try:
    for i in range(len(histovar_list)):
        histobin_list.append(max(list(histo_asc[histovar_list[i]]))+1)
except:
    pass

# create list of histogram units
try:
    histo_units = []
    for i in histovar_list:
        histo_units.append(nc.variables[i].getncattr('units'))
except:
    pass

In [ ]:
try:
    if interactive_histogram == True:
        x_width = 1200
        y_height = 400

        heatmap_data_list = []
        for i in histovar_list: 
            data = histo_asc[i].abs()
            data = data.abs()
            data = data.transpose()
            data = pd.DataFrame(data)
            heatmap_data_list.append(data)
        for i, df in enumerate(heatmap_data_list):
            try:
                max_z = df.to_numpy().max()
            except:
                max_z = 100
            if df.index[0] == 0.0:
                min_y = 1
                ylab = "Bin [#]"
            else:
                min_y = df.index[0]
                ylab = "Cell Size [um]"
            heatmap = df.hvplot.heatmap(cmap="gist_ncar", width=x_width, height=y_height, ylabel=ylab, xlabel='Time [Seconds]', title=histovar_list[i] + ' Units: ' + histo_units[i], logy=True, logz=True, clim=(0.1, max_z), ylim=(min_y, None))            
            bokeh_plot = hv.render(heatmap)
            show(bokeh_plot)
#        # loop over each histogram variable
#        num = 0
#        for i in histovar_list:
#            try:
#                # create the subplot array and size
#                fig, ax = plt.subplots(1,1, figsize=(18, 8))
#                y_axis_labels = list(histo_asc[i].columns)  # labels for y-axis
#                # check to see if Cell Sizes exists (bins always start at 0.0)
#                if (list(histo_asc[i].columns)[0] != 0.0) & (i.startswith('I2DC') == False):
#                    try:
#                        sns.heatmap(histo_asc[i].transpose(), cmap='gist_ncar', cbar_kws={'label': str(histo_units[num])}, norm=LogNorm())
#                    except:
#                        sns.heatmap(histo_asc[i].transpose(), cmap='gist_ncar', cbar_kws={'label': str(histo_units[num])})
#                    sns.heatmap(np.where(histo_asc[i].transpose().isna(), 0, np.nan), cbar=False, fmt="",cmap=ListedColormap(['gray']),linewidth=0)     
#                    plt.ylabel('Cell Size [um]')
#                    # increment the number counter to index into units list when needed
#                    if num < len(histo_units) - 1:
#                        num = num + 1
#                    else:
#                        pass
#                else:
#                    try:
#                        sns.heatmap(histo_asc[i].transpose(), cmap='gist_ncar', cbar_kws={'label': 'Count'}, norm=LogNorm())
#                    except:
#                        sns.heatmap(histo_asc[i].transpose(), cmap='gist_ncar', cbar_kws={'label': 'Count'})
#                    sns.heatmap(np.where(histo_asc[i].transpose().isna(), 0, np.nan),cbar=False,fmt="",cmap=ListedColormap(['gray']),linewidth=0)
#                    plt.ylabel('Bin [#]')
#
#                # check to see if I2DC vars exist, these are inter-arrival times so the y axis label should be Seconds.
#                if i.startswith('I2DC'):
#                    plt.ylabel('Seconds [#]')
#                plt.title(i)
#                plt.xlabel('Time [Seconds]')
#                plt.xticks(fontsize=9, rotation=75)
#                plt.yticks(fontsize=6, rotation=0)
#                ax.invert_yaxis()
#                plt.box(False)
#                plt.yscale('log')
#            except Exception as e:
#                print(e)
#            plt.show()
    else:
        pass
except Exception as e:
    print(e)